In [1]:
import torch
import numpy as np
import random
from IPython import display
from matplotlib import pyplot as plt
import torchvision
import torchvision.transforms as transforms
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"  #防止jupyter崩溃

In [2]:
#下载MNIST手写数据集
mnist_train = torchvision.datasets.MNIST(root='./Datasets/MNIST', train=True,
download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.MNIST(root='./Datasets/MNIST', train=False,
download=True, transform=transforms.ToTensor())
#读取数据
batch_size = 32
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True,
num_workers=0)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False,
num_workers=0)

E:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
#初始化参数
# num_inputs,num_hiddens,num_outputs = 784,256,10
num_inputs,num_hiddens,num_outputs = 784,512,10
W1 = torch.tensor(np.random.normal(0, 0.01, (num_hiddens,num_inputs)), dtype=torch.float32)
b1 = torch.zeros(1, dtype=torch.float32)
W2 = torch.tensor(np.random.normal(0, 0.01, (num_outputs,num_hiddens)), dtype=torch.float32)
b2 = torch.zeros(1, dtype=torch.float32)
params =[W1,b1,W2,b2]
for param in params:
    param.requires_grad_(requires_grad=True)

In [4]:
def l2_penalty(w):
    return (w**2).sum()/2

In [5]:
#定义随机梯度下降法
def SGD(paras,lr):
    for param in params:
        param.data -= lr * param.grad

In [6]:
#定义交叉熵损失函数
loss = torch.nn.CrossEntropyLoss()

In [7]:
#定义模型
def net(X):
    X = X.view((-1,num_inputs))
    H=torch.sigmoid(torch.matmul(X,W1.t())+b1)
    return torch.matmul(H,W2.t())+b2

In [8]:
#测试集loss
def evaluate_loss(data_iter,net):
    acc_sum,loss_sum,n = 0.0,0.0,0
    for X,y in data_iter:
        y_hat = net(X)
        acc_sum += (y_hat.argmax(dim=1)==y).sum().item()
        l = loss(y_hat,y) # l是有关小批量X和y的损失
        loss_sum += l.sum().item()*y.shape[0]
        n+=y.shape[0]
    return acc_sum/n,loss_sum/n

In [9]:
#定义模型训练函数
def train(net,train_iter,test_iter,loss,num_epochs,batch_size,lambd,params=None,lr=None,optimizer=None):
    train_ls = []
    test_ls = []
    for epoch in range(num_epochs): # 训练模型一共需要num_epochs个迭代周期
        train_l_sum, train_acc_num,n = 0.0,0.0,0
        # 在每一个迭代周期中，会使用训练数据集中所有样本一次
        for X, y in train_iter: # x和y分别是小批量样本的特征和标签
            y_hat = net(X)
            #添加惩罚项，用lambd控制惩罚权重，0的话为不使用
            l = loss(y_hat, y)+lambd*(l2_penalty(W1)+l2_penalty(W2)) # l是有关小批量X和y的损失
            l=l.sum()
            #梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward() # 小批量的损失对模型参数求梯度
            if optimizer is None:
                SGD(params,lr)
            else:
                optimizer.step()
            #计算每个epoch的loss
            train_l_sum += l.item()*y.shape[0]
            #计算训练样本的准确率
            train_acc_num += (y_hat.argmax(dim=1)==y).sum().item()
            #每一个epoch的所有样本数
            n+= y.shape[0]
        train_ls.append(train_l_sum/n)
        test_acc,test_l = evaluate_loss(test_iter,net)
        test_ls.append(test_l)
        print('epoch %d, train_loss %.6f,test_loss %f,train_acc %.6f,test_acc %.6f'%(epoch+1, train_ls[epoch],test_ls[epoch],train_acc_num/n,test_acc))
    return train_ls,test_ls

In [12]:
lr = 0.01
num_epochs = 50
train_loss,test_loss = train(net,train_iter,test_iter,loss,num_epochs,batch_size,1,params,lr)

epoch 1, train_loss 7.388573,test_loss 2.303842,train_acc 0.195500,test_acc 0.103200
epoch 2, train_loss 2.308028,test_loss 2.309228,train_acc 0.104183,test_acc 0.095800
epoch 3, train_loss 2.307144,test_loss 2.304580,train_acc 0.103667,test_acc 0.113500
epoch 4, train_loss 2.306595,test_loss 2.310961,train_acc 0.103650,test_acc 0.102800
epoch 5, train_loss 2.305888,test_loss 2.302462,train_acc 0.104100,test_acc 0.113500
epoch 6, train_loss 2.305518,test_loss 2.304229,train_acc 0.105567,test_acc 0.097400
epoch 7, train_loss 2.305011,test_loss 2.303120,train_acc 0.106883,test_acc 0.102800
epoch 8, train_loss 2.304636,test_loss 2.304774,train_acc 0.105133,test_acc 0.102800
epoch 9, train_loss 2.304257,test_loss 2.303165,train_acc 0.108167,test_acc 0.102800
epoch 10, train_loss 2.304026,test_loss 2.302712,train_acc 0.105617,test_acc 0.098000
epoch 11, train_loss 2.303824,test_loss 2.303895,train_acc 0.106217,test_acc 0.113500
epoch 12, train_loss 2.304008,test_loss 2.302258,train_acc 0.10